In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import sklearn.ensemble as ek
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
#from sklearn.externals import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from keras.layers import Input, Dense, Activation, Embedding
from keras.models import Model
from keras.optimizers import SGD
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LinearRegression

In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.shape

(1460, 81)

In [4]:
df=df.drop("Id",axis=1)

In [5]:
df["YearBuilt"].unique()

array([2003, 1976, 2001, 1915, 2000, 1993, 2004, 1973, 1931, 1939, 1965,
       2005, 1962, 2006, 1960, 1929, 1970, 1967, 1958, 1930, 2002, 1968,
       2007, 1951, 1957, 1927, 1920, 1966, 1959, 1994, 1954, 1953, 1955,
       1983, 1975, 1997, 1934, 1963, 1981, 1964, 1999, 1972, 1921, 1945,
       1982, 1998, 1956, 1948, 1910, 1995, 1991, 2009, 1950, 1961, 1977,
       1985, 1979, 1885, 1919, 1990, 1969, 1935, 1988, 1971, 1952, 1936,
       1923, 1924, 1984, 1926, 1940, 1941, 1987, 1986, 2008, 1908, 1892,
       1916, 1932, 1918, 1912, 1947, 1925, 1900, 1980, 1989, 1992, 1949,
       1880, 1928, 1978, 1922, 1996, 2010, 1946, 1913, 1937, 1942, 1938,
       1974, 1893, 1914, 1906, 1890, 1898, 1904, 1882, 1875, 1911, 1917,
       1872, 1905], dtype=int64)

In [6]:
compteur=[]
for i in df.keys():
    if df[i].isnull().sum()>0:
        compteur.append(i)
        print(i,"   ",df[i].isnull().sum())

LotFrontage     259
Alley     1369
MasVnrType     8
MasVnrArea     8
BsmtQual     37
BsmtCond     37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical     1
FireplaceQu     690
GarageType     81
GarageYrBlt     81
GarageFinish     81
GarageQual     81
GarageCond     81
PoolQC     1453
Fence     1179
MiscFeature     1406


In [7]:
#df1=df[df["MasVnrType"]!=]
#df1[["MasVnrType"]]

In [8]:
def Average(df,k):
    df[k].fillna(df[k].mean(),inplace=True)
    return df
def Qualitative(df,k):
    if df[k].isnull().sum()>0:
        df[k].fillna("NA",inplace=True)
    return df

In [9]:
key_numeric=["MasVnrArea","GarageYrBlt",'LotFrontage']
key_qualitatif=["MasVnrType","Alley","BsmtQual","MiscFeature","Fence","PoolQC","FireplaceQu","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","Electrical","GarageType","GarageFinish","GarageQual","GarageCond"]
for i in key_numeric:
    df=Average(df,i)
for i in key_qualitatif:
    df=Qualitative(df,i)   

In [10]:
df.shape

(1460, 80)

In [11]:
#df['MasVnrType']=f
for i in df.keys():
    if df[i].isnull().sum()>0:
        print(i)

In [12]:
df["MasVnrType"].unique()

array(['BrkFace', 'None', 'Stone', 'BrkCmn', 'NA'], dtype=object)

In [13]:
key=["MSSubClass","MSZoning","Alley","PoolQC","Fence","MiscFeature","FireplaceQu","Street","LandContour","LotConfig","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical","Functional","GarageType","SaleType","SaleCondition"]
for i in key:
    lb=LabelEncoder()
    lb.fit(df[i])
    df[i]=lb.transform(df[i])

In [14]:
def OrderQualit(df,i,col):
    lb=LabelEncoder()
    lb.fit(col)
    df[i]=lb.transform(df[i])
    return df

In [15]:
df=OrderQualit(df,'LotShape',['Reg', 'IR1', 'IR2', 'IR3'])
df=OrderQualit(df,'Utilities',['AllPub','NoSeWa'])
df=OrderQualit(df,'LandSlope',['Gtl', 'Mod', 'Sev'])
col=['NA','Po','Fa','TA','Gd','Ex']
df=OrderQualit(df,'ExterQual',col)
df=OrderQualit(df,'ExterCond',col)
df=OrderQualit(df,'BsmtQual',col)
df=OrderQualit(df,'BsmtCond',col)
col1=['NA','No','Mn','Av','Gd']
df=OrderQualit(df,'BsmtExposure',col1)
col2=['NA','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
df=OrderQualit(df,'BsmtFinType1',col2)
df=OrderQualit(df,'BsmtFinType2',col2)
df=OrderQualit(df,'HeatingQC',col)
df=OrderQualit(df,'KitchenQual',col)
col3=['NA','Unf','RFn', 'Fin']
df=OrderQualit(df,'GarageFinish',col3)
df=OrderQualit(df,'GarageQual',col)
df=OrderQualit(df,'GarageCond',col)
col4=['N', 'P','Y']
df=OrderQualit(df,'PavedDrive',col4)

In [16]:
df['PavedDrive'].unique()


array([2, 0, 1])

In [17]:
def Standardisation(X):
    Std_scaler=StandardScaler()
    return Std_scaler.fit_transform(X)

In [18]:
X=df.drop('SalePrice',axis=1)
y=df['SalePrice']
X=Standardisation(X)
X_train, X_test, y_train, y_test =train_test_split(X, y , random_state=10,test_size=0.2)

In [19]:
results = {}
val=100000000
clf =ek.GradientBoostingClassifier(n_estimators=5)
clf.fit(X_train,y_train)
preds= clf.predict(X_test)
score_test = sum((np.log(preds)-np.log(y_test))**2)/len(y_test)
print ("test:",score_test)
preds=clf.predict(X_train)
score_preds=sum((np.log(preds)-np.log(y_train))**2)/len(y_train)
print ("preds:", score_preds)
print ('*'*50)

test: 0.1166968523217564
preds: 0.0017458581017402138
**************************************************
